<a href="https://colab.research.google.com/github/vpavankalyan/FER/blob/main/FER_ResNet50v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/FER/FER.zip", 'r')
zip_ref.extractall("/content/data")
zip_ref.close()

In [ ]:
import tensorflow as tf

In [ ]:
image_size=224
resnet_model = tf.keras.applications.resnet_v2.ResNet50V2(weights='imagenet', include_top=False,input_shape=(image_size, image_size, 3))

94674944/94668760 [==============================] - 1s 0us/step


In [ ]:
for layer in resnet_model.layers:
    layer.trainable = False

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the resnet convolutional base model
model.add(resnet_model)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(7, activation='softmax'))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
# Data Generator for Training data
train_generator = image_gen.flow_from_directory('/content/data/images/train/',
        target_size=(image_size, image_size),
        batch_size=80,
        class_mode='categorical')


Found 28821 images belonging to 7 classes.


In [ ]:
validation_generator = image_gen.flow_from_directory('/content/data/images/validation',
        target_size=(image_size, image_size),
        batch_size=30,
        class_mode='categorical',
        shuffle=False)

Found 7066 images belonging to 7 classes.


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=0.0005),
              metrics=['acc'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)

Epoch 1/30
360/360 [==============================] - 372s 976ms/step - loss: 3.9564 - acc: 0.2415 - val_loss: 1.6060 - val_acc: 0.3742
Epoch 2/30
360/360 [==============================] - 347s 963ms/step - loss: 1.6924 - acc: 0.3429 - val_loss: 1.5586 - val_acc: 0.4185
Epoch 3/30
360/360 [==============================] - 347s 962ms/step - loss: 1.6382 - acc: 0.3676 - val_loss: 1.5417 - val_acc: 0.4249
Epoch 4/30
360/360 [==============================] - 344s 956ms/step - loss: 1.6083 - acc: 0.3835 - val_loss: 1.5251 - val_acc: 0.4328
Epoch 5/30
360/360 [==============================] - 345s 959ms/step - loss: 1.5836 - acc: 0.3987 - val_loss: 1.4936 - val_acc: 0.4338
Epoch 6/30
360/360 [==============================] - 346s 959ms/step - loss: 1.5425 - acc: 0.4076 - val_loss: 1.4896 - val_acc: 0.4408
Epoch 7/30
360/360 [==============================] - 345s 958ms/step - loss: 1.5330 - acc: 0.4106 - val_loss: 1.4660 - val_acc: 0.4720
Epoch 8/30
360/360 [============================

In [ ]:
model.save('/content/drive/MyDrive/FER')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/MyDrive/FER/assets
